In [ ]:
!pip install ncps

In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler


In [ ]:
from google.colab import drive

In [ ]:
drive.mount('/content/drive/')

In [ ]:
trgt=pd.read_csv('/content/drive/MyDrive/Data/la_main.csv')
trgt_test=pd.read_csv('/content/drive/MyDrive/Data/la_main_test.csv')  #import the train and test csv

In [ ]:
columns_to_drop=['Minute', 'Clearsky GHI',
        'Dew Point', 'Solar Zenith Angle','Precipitable Water', 'Fill Flag']

In [ ]:
trgt.drop(columns=columns_to_drop,inplace=True,errors='ignore')
trgt_test.drop(columns=columns_to_drop,inplace=True,errors='ignore')

In [ ]:
trgt=trgt[trgt['GHI']!=0]

In [ ]:
trgt_test=trgt_test[trgt_test['GHI']!=0]

In [ ]:
trgt.drop(trgt.tail(1).index,inplace=True)

In [ ]:
trgt_test.head()

In [ ]:
trgt['Datetime']=pd.to_datetime(trgt[['Year','Month','Day','Hour']])


In [ ]:
trgt.set_index('Datetime',inplace=True)

In [ ]:
trgt.head()

In [ ]:
trgt.sort_index(inplace=True)

In [ ]:
trgt.drop(['Year','Month','Day','Hour'],axis=1,inplace=True)

In [ ]:
trgt_test['Datetime']=pd.to_datetime(trgt_test[['Year','Month','Day','Hour']])

In [ ]:
trgt_test.set_index('Datetime',inplace=True)
trgt_test.sort_index(inplace=True)
trgt_test.drop(['Year','Month','Day','Hour'],axis=1,inplace=True)

In [ ]:
scaler=MinMaxScaler().set_output(transform='pandas')
trgt=scaler.fit_transform(trgt)

In [ ]:
trgt_GHI=trgt[['GHI']].copy()

In [ ]:
def create_sequences(data, sequence_length):
    sequences = []
    targets = []

    for i in range(len(data) - sequence_length):
        seq = data[i:i+sequence_length]
        target = data.iloc[i+sequence_length]
        sequences.append(seq)
        targets.append(target)

    return np.array(sequences), np.array(targets)

In [ ]:
sequence_length=10
_,y_train=create_sequences(trgt_GHI,sequence_length)

In [ ]:
X_train,_=create_sequences(trgt,sequence_length)

In [ ]:
y_train

In [ ]:
scaler1=MinMaxScaler().set_output(transform='pandas')
trgt_test_GHI=scaler1.fit_transform(trgt_test[['GHI']])

In [ ]:
scaler2=MinMaxScaler().set_output(transform='pandas')
trgt_test=scaler2.fit_transform(trgt_test)

In [ ]:
trgt_test.head()

In [ ]:
trgt_test_GHI.head()

In [ ]:
f,y_test=create_sequences(trgt_test_GHI,sequence_length)

In [ ]:
X_test,r=create_sequences(trgt_test,sequence_length)

In [ ]:
import tensorflow as tf
from ncps.tf import LTC
from ncps.tf import CfC

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping


In [ ]:
early_stopping=EarlyStopping(monitor='loss',patience=10,restore_best_weights=True)

In [ ]:
model_2=tf.keras.Sequential([       CfC(64,return_sequences=True),

                                   CfC(32),

                             tf.keras.layers.Dense(1)
                             ]
)  #here, replace with lstm and ltc as per need.

In [ ]:
model_2.compile(loss='mse',optimizer='adam')
model_2.fit(X_train,y_train,epochs=25,callbacks=[early_stopping])

In [ ]:
from sklearn.metrics import mean_squared_error
y_pred=model_2.predict(X_test)

In [ ]:
y_pred=scaler1.inverse_transform(y_pred)
y_test=scaler1.inverse_transform(y_test)

In [ ]:
rmse=np.sqrt(mean_squared_error(y_test,y_pred))
print(rmse)


In [ ]:
from sklearn.metrics import r2_score

In [ ]:
print(r2_score(y_test,y_pred))

In [ ]:
from sklearn.metrics import mean_absolute_error,mean_absolute_percentage_error

In [ ]:
print(mean_absolute_error(y_test,y_pred))
